---
### 도서 : [tensorflow 2 로 자연처리]

#
Tensorflow 
drop out
Dense

In [8]:
## Forrest Park's notion  ## Updated 2024.07.03
### Service  Text Coloring option
from Functions import Service as S
def blue(str):return S.colored_text(str,'blue')
def yellow(str):return S.colored_text(str,'yellow')
def red(str):return S.colored_text(str,'red')
def green(str):return S.colored_text(str,'green')
def imd(str):return S.imd(green(str))
## 자연어처리 패키지 설치 
def NLPInstalls():
    import subprocess,sys
    import warnings ; warnings.filterwarnings('ignore')
    # pip가 없으면 pip를 설치
    try:import pip
    except ImportError:
        print("Install pip for python3")
        subprocess.call(['sudo', 'apt-get', 'install', 'python3-pip'])
    
    # tweepy 없으면 tweepy 설치
    try:import tweepy        
    except ModuleNotFoundError:
        print("Install tweepy")
        subprocess.call([sys.executable, "-m", "pip", "install", 'tweepy==3.10.0'])
    finally:import tweepy 
    
    # konlpy 없으면 konlpy 설치
    try:import konlpy
    except ModuleNotFoundError: 
        print("Install konlpy")
        subprocess.call([sys.executable, "-m", "pip", "install", 'konlpy'])
    finally:import konlpy
    
    # eunjeon 없으면 eunjeon 설치
    try:import eunjeon
    except ModuleNotFoundError: 
        print("Install eunjeon : eunjeon")
        subprocess.call([sys.executable, "-m", "pip", "install", 'eunjeon'])
    finally:import konlpy
    
    # datasets 없으면 datasets를 설치
    try:import datasets
    except ModuleNotFoundError: 
        print("Install datasets : datasets")
        subprocess.call([sys.executable, "-m", "pip", "install", 'datasets'])
    finally:import datasets
    
    # pytorch 없으면 pytorch 설치
    try:import torch
    except ModuleNotFoundError: 
        print("Install torch : pytorch")
        subprocess.call([sys.executable, "-m", "pip", "install", 'pytorch'])
    finally:import torch
    
    # transformers 없으면 transformers 설치
    try:import transformers
    except ModuleNotFoundError: 
        print("Install transformer : transformers")
        subprocess.call([sys.executable, "-m", "pip", "install", 'transformers'])
    finally:import transformers
        
    # UMAP 없으면 UMAP 설치
    try:import umap
    except ModuleNotFoundError: 
        print("Install umap : umap")
        subprocess.call([sys.executable, "-m", "pip", "install", 'umap'])
    finally:import umap
        
    # UMAP 없으면 UMAP 설치
    try:from umap import UMAP
    except ImportError: 
        print("Install umap : umap-learn")
        subprocess.call([sys.executable, "-m", "pip", "install", 'umap-learn'])
    finally:import umap
   
NLPInstalls()
print(yellow(f"📌 - PROGRAM START \n\t"))
## GPU setting (in MacOS)
import torch
print(yellow(f"📌 - MPS 장치를 지원 Build 여부 : "),torch.backends.mps.is_built())
print(yellow(f"📌 - MPS 장치 사용가능 여부 : "),torch.backends.mps.is_available())
print(yellow(f"📌 - GPU 사용Start"))
# device = torch.device("mps")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#### Jupyter Basic Setting ####
import pandas as pd,numpy as np
import matplotlib.pyplot as plt,seaborn as sns  # 시각화
import warnings; warnings.filterwarnings('ignore')  # 경고 무시
import sys,os 
from pprint import pprint


📌 - PROGRAM START 
	
📌 - MPS 장치를 지원 Build 여부 :  True
📌 - MPS 장치 사용가능 여부 :  True
📌 - GPU 사용Start


---
### Tensor flow 간단한 자연어 처리 예시 01 page 53

In [32]:
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras import preprocessing 


samples = [' 너 오늘 이뻐 보인다',
           '나는 오늘 기분이 더러워 ',
           '끝내주는데, 좋은 일이 있나봐',
           '나 좋은 일이 생겼어',
           '아 오늘 진짜 짜증나',
           '환상적인데, 정말 좋은 거 같아 ',
           ]
## labeling 
labels = [[1],[0],[1],[1],[0],[1]]
## tokenize
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(samples)

sequences = tokenizer.texts_to_sequences(samples)
print(yellow(f"📌 - text =>sequences"))
pprint([(f"{i} => {j}") for i,j in zip(samples,sequences)])
word_index = tokenizer.word_index
print(yellow(f"\n📌 - word index"))
pprint(word_index)


📌 - text =>sequences
[' 너 오늘 이뻐 보인다 => [4, 1, 5, 6]',
 '나는 오늘 기분이 더러워  => [7, 1, 8, 9]',
 '끝내주는데, 좋은 일이 있나봐 => [10, 2, 3, 11]',
 '나 좋은 일이 생겼어 => [12, 2, 3, 13]',
 '아 오늘 진짜 짜증나 => [14, 1, 15, 16]',
 '환상적인데, 정말 좋은 거 같아  => [17, 18, 2, 19, 20]']

📌 - word index
{'같아': 20,
 '거': 19,
 '기분이': 8,
 '끝내주는데': 10,
 '나': 12,
 '나는': 7,
 '너': 4,
 '더러워': 9,
 '보인다': 6,
 '생겼어': 13,
 '아': 14,
 '오늘': 1,
 '이뻐': 5,
 '일이': 3,
 '있나봐': 11,
 '정말': 18,
 '좋은': 2,
 '진짜': 15,
 '짜증나': 16,
 '환상적인데': 17}


---
### 추가로 모델 구축 및 모델 학습에 피요한 변수 정의 

In [34]:
batch_size = 2
num_epochs = 100
vocab_size =len(word_index) +1
emb_size = 128
hidden_dimension = 256
output_dimension = 1


---
### 데이터를 통과 시킬 모델 구현

In [36]:
model = tf.keras.Sequential()
model.add(layers.Embedding(vocab_size,emb_size,input_length= 4))
model.add(layers.Lambda(lambda x: tf.reduce_mean(x,axis= 1)))
model.add(layers.Dense(hidden_dimension, activation='relu'))
model.add(layers.Dense(output_dimension,activation ="sigmoid"))